In [4]:
import os
import psycopg2
from dotenv import load_dotenv
from rich.console import Console
from rich.table import Table
import urllib.parse

# Initialize console for rich output
console = Console()

def get_product_families():
    """
    Connect to the BlackPearl database and retrieve all product families,
    including sample products with their brand names.

    Returns:
        List of dictionaries containing product family information
    """
    # Load environment variables
    load_dotenv()

    # Get database URI from environment
    db_uri = os.getenv('BLACKPEARL_DB_URI')

    if not db_uri:
        console.print("[red]Error: BLACKPEARL_DB_URI not found in environment variables[/red]")
        return []

    # Parse the connection string
    result = urllib.parse.urlparse(db_uri)
    username = result.username
    password = result.password
    database = result.path[1:]
    hostname = result.hostname
    port = result.port

    console.print(f"[blue]Connecting to database {database} at {hostname}...[/blue]")

    conn = None  # Initialize conn to None
    try:
        # Connect to the database
        conn = psycopg2.connect(
            host=hostname,
            port=port,
            database=database,
            user=username,
            password=password,
            connect_timeout=10
        )

        # Create a cursor
        cursor = conn.cursor()

        # Query product families
        cursor.execute("""
            SELECT id, codigo, "nomeFamilia"
            FROM catalogo_familiadeproduto
            ORDER BY "nomeFamilia"
        """)

        families = cursor.fetchall()

        # Fetch all brands into a dictionary for quick lookup {brand_id: brand_name}
        cursor.execute("SELECT id, nome FROM catalogo_marca")
        brands_dict = {brand[0]: brand[1] for brand in cursor.fetchall()}


        # Convert to list of dictionaries
        result = []
        for family in families:
            family_id = family[0]
            family_name = family[2] # "nomeFamilia"

            # Get 10 sample products for each family, including brand name
            # Joining catalogo_produto (aliased p) with catalogo_marca (aliased m)
            cursor.execute("""
                SELECT p.id, p.descricao, p.descr_detalhada, p.ean, p.marca_id
                FROM catalogo_produto p
                WHERE p.familia_id = %s
                LIMIT 10
            """, (family_id,))

            sample_products_raw = cursor.fetchall()

            # Map raw product data to include brand name from the brands_dict
            products_list = []
            for p in sample_products_raw:
                brand_id = p[4]
                brand_name = brands_dict.get(brand_id, "Marca Desconhecida") # Lookup brand name
                products_list.append({
                    "id": p[0],
                    "nome": p[1], # from p.descricao
                    "descricao_detalhada": p[2], # from p.descr_detalhada
                    "ean": p[3], # from p.ean
                    "marca": brand_name # Fetched brand name
                })


            result.append({
                "id": family_id,
                "codigo": family[1],
                "nome_familia": family_name,
                "sample_products": products_list # Now includes brand name
            })

        # Display the results in a table (only family info)
        table = Table(title="Product Families")
        table.add_column("ID", style="cyan")
        table.add_column("Code", style="green")
        table.add_column("Family Name", style="yellow")

        for family_data in result:
            table.add_row(
                str(family_data["id"]),
                str(family_data["codigo"]),
                family_data["nome_familia"]
            )

        console.print(table)
        console.print(f"\n[green]Total product families: {len(result)}[/green]")

        # Close cursor and connection
        cursor.close()
        conn.close()

        return result

    except Exception as e:
        console.print(f"[red]Error during database operation:[/red] {str(e)}")
        # Ensure connection is closed even if an error occurs
        if conn:
            conn.close()
        return []

product_families = get_product_families()
print(product_families)

Connecting to database solidpower_db at db-postgresql-theros-do-user-19117798-0.g.db.ondigitalocean.com...

               Product Families               
┏━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ ID  ┃ Code        ┃ Family Name            ┃
┡━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 115 │ 0           │                        │
│ 65  │ 1129682242  │ ACESSÓRIOS GABINETE    │
│ 113 │ 10546186400 │ ACESSÓRIOS OFFICE      │
│ 109 │ 10506555149 │ ADAPTADOR USB WIFI     │
│ 81  │ 1129682258  │ AIR COOLER             │
│ 85  │ 1129682262  │ APOIO PARA TECLADO     │
│ 104 │ 7630787841  │ AUTO CARE              │
│ 93  │ 1643432804  │ BERMUDA                │
│ 84  │ 1129682261  │ BOARD GAMES            │
│ 96  │ 6759916857  │ CABOS                  │
│ 88  │ 1163533626  │ CADEIRA                │
│ 83  │ 1129682260  │ CAIXA DE SOM           │
│ 90  │ 1397365053  │ CAMISETA               │
│ 91  │ 1397365071  │ CAMISOLA               │
│ 82  │ 1129682259  │ CANETA DIGITAL         │
│ 61  │ 1129682238  │ COMBO                  │
│ 86  │ 1129682263  │ CONSOLE                │
│ 99  │ 7520871400  │ DAC                    │
│ 112 │ 10506555811 │ DOCK STATION           │
│ 70  │ 1129682247  │ FAN                    │
│ 80  │ 1129682257  │ FONTE                  │
│ 79  │ 1129682256  │ GABINETE               │
│ 78  │ 1129682255  │ HEADSET                │
│ 111 │ 10506555697 │ HUB/SWITCH             │
│ 102 │ 7561382514  │ ILUMINAÇÃO             │
│ 77  │ 1129682254  │ JOYSTICK               │
│ 60  │ 1129682237  │ KEYCAPS                │
│ 101 │ 7537275456  │ MEMÓRIA                │
│ 76  │ 1129682253  │ MESA DIGITALIZADORA    │
│ 75  │ 1129682252  │ MICROFONE              │
│ 87  │ 1129682264  │ MOBILE                 │
│ 74  │ 1129682251  │ MOCHILA                │
│ 73  │ 1129682250  │ MONITORES              │
│ 72  │ 1129682249  │ MOUSE                  │
│ 71  │ 1129682248  │ MOUSEPAD               │
│ 114 │ 10597965118 │ NOBREAK                │
│ 69  │ 1129682246  │ PASTA TÉRMICA          │
│ 107 │ 10455298528 │ PC CARE                │
│ 94  │ 1696045999  │ PELÚCIA                │
│ 105 │ 10444013670 │ PEN DRIVE              │
│ 89  │ 1241237250  │ PIJAMA                 │
│ 98  │ 7520871001  │ PLACA DE SOM           │
│ 103 │ 7627068421  │ PLACA DE VÍDEO         │
│ 68  │ 1129682245  │ PLACA MÃE              │
│ 110 │ 10506555482 │ ROTEADOR WIFI          │
│ 100 │ 7537275412  │ SSD                    │
│ 67  │ 1129682244  │ STARCHEF               │
│ 106 │ 10448236085 │ STREAMER               │
│ 66  │ 1129682243  │ SUPORTE PARA HEADSET   │
│ 108 │ 10458435644 │ SUPORTE PARA MICROFONE │
│ 92  │ 1515075769  │ SUPORTE PARA NOTEBOOK  │
│ 95  │ 6759916663  │ SWITCH                 │
│ 97  │ 6763892705  │ TECLADO DIY            │
│ 64  │ 1129682241  │ TECLADO MECANICO       │
│ 62  │ 1129682239  │ TECLADO MEMBRANA       │
│ 63  │ 1129682240  │ TECLADO OPTICO         │
│ 59  │ 1129682236  │ WATER COOLER           │
│ 58  │ 1129682235  │ WEBCAM                 │
└─────┴─────────────┴────────────────────────┘

Total product families: 58

[{'id': 115, 'codigo': 0, 'nome_familia': '', 'sample_products': [{'id': 4571, 'nome': 'MAQUINA RHEA XS E/3', 'descricao_detalhada': None, 'ean': '', 'marca': ''}, {'id': 4500, 'nome': 'VOLVO/FH 420 4X2T PLACA QHA8A34 2014/2014', 'descricao_detalhada': None, 'ean': '', 'marca': ''}, {'id': 4531, 'nome': 'Fonte de energia para carga direta para coletores Bluebird EF400, EF500 e Tablet RT100', 'descricao_detalhada': None, 'ean': '', 'marca': ''}, {'id': 4532, 'nome': 'Capa Nacional de Protecao para Coletor BM180 BLUEBIRD com bateria padrao (Com alca de ombro) - BR', 'descricao_detalhada': None, 'ean': '', 'marca': ''}, {'id': 4533, 'nome': 'Cabo Micro USB', 'descricao_detalhada': None, 'ean': '', 'marca': ''}, {'id': 4590, 'nome': 'FILL AIR ROCKET TOP ASSY - 220 VAC', 'descricao_detalhada': None, 'ean': '', 'marca': ''}, {'id': 4596, 'nome': 'TECLADO MECANICO RAHU RGB SWITCH AZUL (K567 RGB) - REDRAGON', 'descricao_detalhada': None, 'ean': '6950376750747', 'marca': ''}, {'id': 4705, 'nome

In [5]:
product_families

[{'id': 115,
  'codigo': 0,
  'nome_familia': '',
  'sample_products': [{'id': 4571,
    'nome': 'MAQUINA RHEA XS E/3',
    'descricao_detalhada': None,
    'ean': '',
    'marca': ''},
   {'id': 4500,
    'nome': 'VOLVO/FH 420 4X2T PLACA QHA8A34 2014/2014',
    'descricao_detalhada': None,
    'ean': '',
    'marca': ''},
   {'id': 4531,
    'nome': 'Fonte de energia para carga direta para coletores Bluebird EF400, EF500 e Tablet RT100',
    'descricao_detalhada': None,
    'ean': '',
    'marca': ''},
   {'id': 4532,
    'nome': 'Capa Nacional de Protecao para Coletor BM180 BLUEBIRD com bateria padrao (Com alca de ombro) - BR',
    'descricao_detalhada': None,
    'ean': '',
    'marca': ''},
   {'id': 4533,
    'nome': 'Cabo Micro USB',
    'descricao_detalhada': None,
    'ean': '',
    'marca': ''},
   {'id': 4590,
    'nome': 'FILL AIR ROCKET TOP ASSY - 220 VAC',
    'descricao_detalhada': None,
    'ean': '',
    'marca': ''},
   {'id': 4596,
    'nome': 'TECLADO MECANICO RAHU R

In [27]:
import asyncio
from typing import Dict, Any, List
from pydantic import BaseModel, Field
from pydantic_ai import Agent
import json
from tqdm import tqdm

class ProductFamilySpecSchema(BaseModel):
    """Schema definition for a product family's specifications."""
    family_id: int = Field(description="ID of the product family")
    family_name: str = Field(description="Name of the product family")
    specs_schema: Dict[str, Dict[str, Any]] = Field(
        description="JSON schema for the specifications of this product family"
    )

async def generate_family_spec_schema(family_id: int, family_name: str) -> ProductFamilySpecSchema:
    """
    Generate a specification schema for a given product family.
    
    Args:
        family_id: ID of the product family
        family_name: Name of the product family
        
    Returns:
        A structured schema for the product family's specifications
    """
    spec_schema_generator = Agent(
        'openai:o3-mini',
        result_type=ProductFamilySpecSchema,
        retries=3,
        system_prompt="""
        You are a product specification schema expert. Your task is to create a JSON schema that defines 
        the structure of specifications for a specific product family in the computer hardware and peripherals domain.
        
        Guidelines:
        - Use snake_case for all keys
        - Define appropriate data types for each specification (string, integer, float, boolean, array, object)
        - For each field, include:
          * type: The data type
          * description: A brief description in Portuguese
          * format (if applicable): Any specific format requirements
          * enum (if applicable): Possible values for enumerated types
        - Focus on specifications that are commonly used for comparison and decision-making
        - Use Portuguese for all descriptions and enum values
        - Keep the schema concise but comprehensive enough for meaningful product comparisons
        
        The schema should follow this pattern for each specification:
        
        {
          "spec_name": {
            "type": "string|integer|number|boolean|array|object",
            "description": "Description in Portuguese",
            "format": "Optional format specification",
            "enum": ["Value1", "Value2"] // If applicable
          }
        }
        
        For example, for a mouse, one field might be:
        
        {
          "dpi_maximo": {
            "type": "integer",
            "description": "Resolução máxima do sensor em DPI (Dots Per Inch)",
            "minimum": 100
          }
        }
        """
    )
    
    # Create a prompt that includes the family information
    prompt = f"""
    Generate a specification schema for the product family:
    
    Family ID: {family_id}
    Family Name: {family_name}
    
    Create a comprehensive but practical schema that captures the most important 
    specifications for this type of product. Focus on specifications that consumers 
    would use to compare products and make purchasing decisions.
    """
    
    result = await spec_schema_generator.run(prompt)
    return result.data

async def process_all_families(families: List[Dict[str, Any]]) -> List[ProductFamilySpecSchema]:
    """Process all product families to generate specification schemas."""
    results = []
    
    # Create a progress bar
    for family in tqdm(families, desc="Processing families", unit="family"):
        # Skip empty family names
        if not family["nome_familia"]:
            continue
            
        try:
            print(f"Processing family: {family['nome_familia']} (ID: {family['id']})")
            schema = await generate_family_spec_schema(family["id"], family["nome_familia"])
            results.append(schema)
            print(f"Family Schema: {schema}")
            # Save intermediate results to avoid losing progress
            with open(f"family_spec_{family['id']}.json", "w", encoding="utf-8") as f:
                json.dump(schema.model_dump(), f, ensure_ascii=False, indent=2)
                
        except Exception as e:
            print(f"Error processing family {family['nome_familia']}: {str(e)}")
    
    # Save complete results
    with open("all_solid_family_specs.json", "w", encoding="utf-8") as f:
        json.dump([r.model_dump() for r in results], f, ensure_ascii=False, indent=2)
        
    return results


In [28]:
# Process a subset of families for testing (comment this out to process all)
test_families = product_families[:10]  
family_specs = await process_all_families(test_families)


# Uncomment to process all families
# family_specs = process_all_families(product_families)

Processing family: ACESSÓRIOS GABINETE (ID: 65)
Processing family: ACESSÓRIOS OFFICE (ID: 113)
Processing family: ADAPTADOR USB WIFI (ID: 109)
Processing family: AIR COOLER (ID: 81)
Processing family: APOIO PARA TECLADO (ID: 85)
Processing family: AUTO CARE (ID: 104)
Processing family: BERMUDA (ID: 93)


CancelledError: 

In [23]:
# Display the results
print(f"\nProcessed {len(family_specs)} product families")
for spec in family_specs:
    print(f"\nFamily: {spec.family_name} (ID: {spec.family_id})")
    print(f"Number of specifications: {len(spec.specs_schema)}")
    print("Sample specifications:")
    sample_specs = list(spec.specs_schema.items())[:3]  # Show first 3 specs
    for name, details in sample_specs:
        print(f"  - {name}: {details['type']} ({details['description'][:50]}...)")
        



Processed 2 product families

Family: ACESSÓRIOS GABINETE (ID: 65)
Number of specifications: 10
Sample specifications:
  - material_construcao: string (Material de construção do acessório do gabinete...)
  - cor: string (Cor do acessório do gabinete...)
  - peso: number (Peso do acessório do gabinete em gramas...)

Family: ACESSÓRIOS OFFICE (ID: 113)
Number of specifications: 8
Sample specifications:
  - material: string (Material do qual o acessório é feito...)
  - cor: string (Cor do acessório...)
  - peso: number (Peso do acessório em gramas...)


In [9]:
import asyncio
import os
import json
from typing import Dict, Any, List, Optional
from pydantic_ai import Agent, RunContext, Tool
import logging
from pydantic import BaseModel, Field
from src.config import settings
from pydantic_ai.common_tools.tavily import tavily_search_tool

tavily_api_key = os.getenv('TAVILY_API_KEY')
assert tavily_api_key is not None

logger = logging.getLogger(__name__)

class ProductInfo(BaseModel):
    """Information extracted about a product."""
    nome: str = Field(description="Nome do produto")
    marca: str = Field(description="Marca do produto")
    especificacoes: Dict[str, Any] = Field(description="Especificações técnicas do produto em formato estruturado")
    descricao: str = Field(description="Descrição geral do produto")
    categoria: str = Field(description="Categoria do produto")
    imagens: List[str] = Field(description="Lista de URLs das imagens do produto")


async def extract_product_info(product_name: str) -> ProductInfo:
    """
    Extract structured information about a product using Tavily search and LLM processing.
    
    Args:
        product_name: Name of the product to search for
        
    Returns:
        Structured product information
    """
    product_extractor = Agent(
        'openai:gpt-4o',
        result_type=ProductInfo,
        system_prompt="""
        You are a product information extraction specialist. Your task is to extract detailed, structured information 
        about computer hardware and peripherals from search results.
        
        Guidelines:
        - Focus on extracting accurate technical specifications
        - Use snake_case for specification keys
        - Structure the data according to the product type (mouse, keyboard, monitor, etc.)
        - Use appropriate data types (string, integer, float, boolean, array)
        - Prefer standardized values where possible
        - Use Portuguese for keys and values
        - Images URL should be in format of a list of URLs
        
        For keyboards, include details like:
        - formato (TKL, Full-Size, etc.)
        - switch_tipo (Linear, Tátil, Clicky)
        - switch_marca (Cherry, Gateron, etc.)
        - conectividade (USB, Bluetooth, etc.)
        
        For mice, include details like:
        - sensor_tipo (Óptico, Laser)
        - dpi_maximo (integer value)
        - ergonomia (Destro, Canhoto, Ambidestro)
        - conectividade (USB, Bluetooth, etc.)
        
        For monitors, include details like:
        - tamanho_tela_polegadas (float value)
        - resolucao_pixels (e.g., "2560x1440")
        - tipo_painel (IPS, VA, TN, etc.)
        - taxa_atualizacao_hz (integer value)
        
        Extract as much relevant information as possible from the search results to create a comprehensive 
        product specification.
        """,
        tools=[tavily_search_tool(tavily_api_key)]
    )
    
    result = await product_extractor.run(
        f"Extract detailed information about this product: {product_name}"
    )
    
    return result.data


In [10]:
product_name = "Teclado Mecânico Redragon Kumara K552 RGB"
product_info = await extract_product_info(product_name)
print(json.dumps(product_info.model_dump(), indent=2, ensure_ascii=False))


{
  "nome": "Redragon Kumara K552 RGB",
  "marca": "Redragon",
  "especificacoes": {
    "formato": "TKL",
    "switch_tipo": "Clicky",
    "switch_marca": "Outemu Blue",
    "conectividade": "USB"
  },
  "descricao": "O Redragon Kumara K552 RGB é um teclado mecânico compacto Tenkeyless (TKL) projetado para jogos. Possui switches táteis clicky Outemu Blue que são conhecidos por seu som audível de clique e resistência tátil ao pressionar as teclas. O teclado tem iluminação RGB individual por tecla, permitindo uma personalização extensa das luzes. É construído com uma base de liga de metal e ABS que garante durabilidade em condições de jogo intensas. O teclado conta com tecnologia anti-ghosting em todas as 87 teclas e um conector USB banhado a ouro para uma conexão confiável.",
  "categoria": "Teclado Mecânico",
  "imagens": [
    "https://redragonshop.com/products/redragon-kumara-k552?srsltid=AfmBOoojFotTs7UXVxwT2EEGFjxIAH3jDPRcgs6SLnotkS-JD0VbYE8J",
    "https://api.bestbuy.com/click/-